In [2]:
pip install -q datasets transformers[torch] evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
dataset = load_dataset("glue", "mrpc")
dataset

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained( "bert-base-uncased")
mapped_dataset = dataset.map(lambda x: tokenizer(x["sentence1"], x["sentence2"]), batched = True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments
import evaluate
import numpy as np
def compute_metrics(eval_preds):
  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis =-1)
  return metric.compute(predictions=predictions, references = labels)
#train_args = TrainingArguments("test-run-1", evaluation_strategy= "epoch")

from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir='./results',                   # directory to save results
    evaluation_strategy="epoch",              # evaluation at the end of each epoch
    learning_rate=1e-2,                      # learning rate
    per_device_train_batch_size=16,           # batch size for training
    weight_decay=0.01,                        # strength of weight decay
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,
    seed=42,                                   # random seed for reproducibility
    adam_beta1=0.9,                           # beta1 for Adam optimizer
    adam_beta2=0.999,                         # beta2 for Adam optimizer
    adam_epsilon=1e-08,                       # epsilon for Adam optimizer
    lr_scheduler_type="cosine_with_restarts",
    num_train_epochs=10,
    warmup_steps=500,
    gradient_accumulation_steps=2
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [9]:
pip install loralib

In [20]:
import loralib as lora
def apply_lora_to_bert(bert_model, r=4, lora_alpha=8, lora_dropout=0.1):
    for layer in bert_model.bert.encoder.layer:
        # Apply LoRA to the query and key projection layers
        layer.attention.self.query = lora.Linear(
            layer.attention.self.query.in_features,
            layer.attention.self.query.out_features,
            r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
        )
        layer.attention.self.key = lora.Linear(
            layer.attention.self.key.in_features,
            layer.attention.self.key.out_features,
            r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
        )
        layer.attention.self.value = lora.Linear(
            layer.attention.self.value.in_features,
            layer.attention.self.value.out_features,
            r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
        )
    return bert_model

# Apply LoRA to the BERT model's attention layers
model = apply_lora_to_bert(model)

In [21]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
              )
              (key): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
              )
              (value): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout)

In [22]:
lora.mark_only_lora_as_trainable(model)

In [23]:
def count_trainable_params(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return trainable_params
def show_trainable_params(model):
    print("Trainable parameters (requires_grad=True):")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.shape}")


print(count_trainable_params(model))
show_trainable_params(model)

221184
Trainable parameters (requires_grad=True):
bert.encoder.layer.0.attention.self.query.lora_A: torch.Size([4, 768])
bert.encoder.layer.0.attention.self.query.lora_B: torch.Size([768, 4])
bert.encoder.layer.0.attention.self.key.lora_A: torch.Size([4, 768])
bert.encoder.layer.0.attention.self.key.lora_B: torch.Size([768, 4])
bert.encoder.layer.0.attention.self.value.lora_A: torch.Size([4, 768])
bert.encoder.layer.0.attention.self.value.lora_B: torch.Size([768, 4])
bert.encoder.layer.1.attention.self.query.lora_A: torch.Size([4, 768])
bert.encoder.layer.1.attention.self.query.lora_B: torch.Size([768, 4])
bert.encoder.layer.1.attention.self.key.lora_A: torch.Size([4, 768])
bert.encoder.layer.1.attention.self.key.lora_B: torch.Size([768, 4])
bert.encoder.layer.1.attention.self.value.lora_A: torch.Size([4, 768])
bert.encoder.layer.1.attention.self.value.lora_B: torch.Size([768, 4])
bert.encoder.layer.2.attention.self.query.lora_A: torch.Size([4, 768])
bert.encoder.layer.2.attention.self

In [24]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = train_args,
    train_dataset = mapped_dataset["train"],
    eval_dataset = mapped_dataset["validation"],
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.605000,0.617748,0.700980,0.817365
2,0.658700,0.628044,0.683824,0.812227
3,0.645400,0.624117,0.683824,0.812227
4,0.668500,0.626570,0.683824,0.812227
5,0.636100,0.623987,0.683824,0.812227
6,0.632200,0.624126,0.683824,0.812227
7,0.634500,0.623990,0.683824,0.812227
8,0.607100,0.623969,0.683824,0.812227
9,0.618600,0.623960,0.683824,0.812227
10,0.665300,0.623955,0.683824,0.812227


TrainOutput(global_step=1150, training_loss=0.6334076553842296, metrics={'train_runtime': 419.9254, 'train_samples_per_second': 87.349, 'train_steps_per_second': 2.739, 'total_flos': 1436395093366032.0, 'train_loss': 0.6334076553842296, 'epoch': 10.0})

In [26]:
import numpy
preds = trainer.predict(mapped_dataset["test"])
logits, labels, _ = preds
predictions = numpy.argmax(logits, axis =-1)
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions = predictions, references = labels)

{'accuracy': 0.664927536231884, 'f1': 0.7987465181058496}